In [18]:
import tweepy
import joblib
from wordcloud import WordCloud
from tweepy import OAuthHandler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
plt.style.use('fivethirtyeight')

# Global Parameters
stop_words = set(stopwords.words('english'))

### Cargamos el modelo Logistic Regression

In [19]:
# load the model from disk
loaded_model = joblib.load('lr_model.joblib')

### Se cargan las claves para acceder a la API de Twitter

In [20]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'rKyttCwUrUgz6JFyJMGPzLWWi'
consumer_secret = 'b7k9Mi0iNkFHJtEd3KDFSgb86NdhI47uIxCTKA687F0TXh4n6r'
access_token = '1394615213199081475-xDnRhKd4El8p9MjjqGr7uYJ32ckx6q'
access_token_secret = 'YpPRz14y3KcpnwK7tE67wMWWCFKC5lJyp62ZIYN8r3Voh'

username = "MinPrac"

# attempt authentication
try:
    # create OAuthHandler object
    auth = OAuthHandler(consumer_key, consumer_secret)
    # set access token and secret
    auth.set_access_token(access_token, access_token_secret)
    # create tweepy API object to fetch tweets
    api = tweepy.API(auth)
except:
    print("Error: Authentication Failed")

### Se cargan los ultimos 10 tweets del usuario indicado y se muestran los 5 mas recientes

In [21]:
# Extract 10 tweets from the twitter user
posts = api.user_timeline(screen_name=username, count = 10, lang ="en", tweet_mode="extended")

#  Print the last 5 tweets
print("Show the 5 recent tweets:\n")
i=1
for tweet in posts[:5]:
    print(str(i) +') '+ tweet.full_text + '\n')
    i= i+1

Show the 5 recent tweets:

1) I was sad

2) Keep up the good work!

3) I am very happy with the news

4) Today has been a great day for all the retail investors

5) I need a break



### Creamos el dataframe con los tweets cargados en el paso anterior

In [22]:
# Create a dataframe with a column called Tweets
df = pd.DataFrame([tweet.full_text for tweet in posts], columns=['Tweets'])
# Show the first 5 rows of data
df.head()

,Tweets
0,I was sad
1,Keep up the good work!
2,I am very happy with the news
3,Today has been a great day for all the retail ...
4,I need a break


### Limpieza del texto
Hacemos una limpia del texto del tweet. Se eliminan las URLs, todas las referencias, los hashtags, las puntuaciones, eliminar las stopwords y por ultimo usamos la técnica de Lematización.
<br/><br/>
La lematización es un proceso lingüístico que consiste en, dada una forma flexionada (es decir, en plural, en femenino, conjugada, etc), hallar el lema correspondiente.
<br/><br/>
![title](lematizacion.png)

In [23]:
def preprocess_tweet_text(tweet):
    tweet.lower()
    # Remove urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from tweet
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove punctuations
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stop_words]
    
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    
    return " ".join(lemma_words)

df['Tweets'] = df['Tweets'].apply(preprocess_tweet_text)
df

,Tweets
0,I sad
1,keep good work
2,I happi news
3,today great day retail investor
4,I need break
5,I hungri
6,feel devast


### Usamos TFidf Vectorizer
TF-IDF (Frecuencia de documento de frecuencia de inversión de término) es una técnica de ponderación comúnmente utilizada en el procesamiento de información y la minería de datos. Esta técnica utiliza un método estadístico para calcular la importancia de una palabra en todo el corpus en función del número de veces que la palabra aparece en el texto y la frecuencia de los documentos que aparecen en todo el corpus. Su ventaja es que puede filtrar algunas palabras comunes pero irrelevantes, mientras retiene palabras importantes que afectan todo el texto.
<br/></br>
TF (Frecuencia de término) indica con qué frecuencia aparece una palabra clave en todo el artículo.
<br/>
IDF (Frecuencia de documento de inversión) significa calcular la frecuencia de texto invertida. La frecuencia del texto se refiere al número de veces que aparece una palabra clave en todos los artículos de todo el corpus. La frecuencia de documento invertida también se denomina frecuencia de documento inversa. Es el recíproco de la frecuencia de documento. Se utiliza principalmente para reducir el efecto de algunas palabras comunes en todos los documentos que tienen poco efecto en el documento.

In [24]:
def get_feature_vector(train_fit):
    vector = joblib.load('tf_model.joblib')
    vector.transform(train_fit)
    return vector

# Same tf vector will be used for Testing sentiments on unseen trending data
tf_vector = get_feature_vector(np.array(df.iloc[:]).ravel())
X = tf_vector.transform(np.array(df.iloc[:]).ravel())

### Predicción del modelo cargado anteriormente (Logistic Regression)


In [25]:
predicted_sentiments = loaded_model.predict(X)
df['Sentiment'] = ['negative' if sentiment == 0 else 'positive' for sentiment in predicted_sentiments]
df

,Tweets,Sentiment
0,I sad,negative
1,keep good work,positive
2,I happi news,positive
3,today great day retail investor,positive
4,I need break,negative
5,I hungri,negative
6,feel devast,negative


### Preparación del CSV para luego cargarlo en el recomendador de peliculas

In [26]:
# Preparing CSV files

user = api.get_user(username)
user_id = user.id_str

positiveTweets = 0
negativeTweets = 0
for sentiment in df['Sentiment']:
    if sentiment == 'positive':
        ++positiveTweets
    else:
        ++negativeTweets

status = 'positive' if positiveTweets > negativeTweets else 'negative'

diccionario = {'user_id': [user_id], 'user_name': [username], 'status': [status]}

df_user = pd.DataFrame(diccionario)

df_user.to_csv('userSentiment.csv', header=True, index=False)